# Shielding Plots for the CosmicWatch Muon Detector

This script can be used to make some simple plots using iPython.
You'll require some imports, all of which are in the next cell.

This script is adapted from the Example iPython Notebook provided in the original CosmicWatch v2 GitHub.

In [ ]:
%pip install numpy matplotlib ipython spicy

In [ ]:
#***************
# Master import
#***************

# Plot data in the window
%matplotlib inline
# Make the cell width the size of the window
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Imports
import glob
import sys
#import nplot as NPlot ##added
import pylab
from pylab import *
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy as sp
import time
import numpy as np
import os

# Fontsize
font = { 'size':   24}
matplotlib.rc('font', **font)

'''
# Allow it to use Latex notation
plt.rc('text', usetex=True)
'''
# Define your own colors for the plots
mycolors = ['#c70039','#ff5733','#ff8d1a','#ffc300','#eddd53','#add45c','#57c785',
               '#00baad','#2a7b9b','#3d3d6b','#511849','#900c3f','#900c3f'] 

#***************
# Master import
#***************
print('Imports complete ...')

In [ ]:
# ***********************************************************************************
# CWClass
# This class is used to import CW data and use it easily. It will use either the 
# data recorded by the computer or from the microSD Card.
#
# Arguments:
#    1. fname: 
#       location and name of the file that you want to plot
#    2. bin_size:
#       When plotting rate as a function of time, you need to specify the rate over
#       a given interval. "bin_size" is the interval in seconds. Default is 60s.
#***********************************************************************************

class CWClass():
    def __init__(self,fname,bin_size = 60):
        self.name = fname.split('/')[-1]
        self.bin_size = bin_size
        
        fileHandle = open(fname,"r" )
        lineList = fileHandle.readlines()
        fileHandle.close()
        header_lines = 0
        
        # Look through the first 1000 lines for the word "Device". Everything prior is considered part of the header.
        for i in range(min(len(lineList),1000)):
            header_in_file = lineList[i]
            if 'Device' in header_in_file:
                header_lines = i+1
  
        #Determine number of columns by looking at the second last line in the file.
        number_of_columns = len(lineList[len(lineList)-2].split(" "))
        column_array = range(0,number_of_columns)

        
        file_from_computer = False
        file_from_sdcard   = False
        
        if number_of_columns == 9:
            file_from_computer = True  # If you have 9 columns, you probably took the data on the computer 
            data = np.genfromtxt(fname, dtype = str, delimiter=' ', usecols=column_array, invalid_raise=False, skip_header=header_lines)
            comp_date = data[:,0] #first column of data
            comp_time = data[:,1]
            event_number = data[:,2].astype(float) #replace np.float w just float 
            Ardn_time_ms = data[:,3].astype(float) #
            adc = data[:,4].astype(float)#
            sipm = data[:,5].astype(float)#
            deadtime = data[:,6].astype(float)#
            temperature = data[:,7].astype(float)#
            detName = data[:,8]
            
        elif number_of_columns == 6:
            file_from_sdcard = True # If you have 6 columns, you took the data from the sdCard
            data = np.genfromtxt(fname, dtype = str, delimiter=' ', usecols=column_array, invalid_raise=False, skip_header=header_lines)
            event_number = data[:,0].astype(float)#
            Ardn_time_ms = data[:,1].astype(float)
            adc = data[:,2].astype(float)
            sipm = data[:,3].astype(float)
            deadtime = data[:,4].astype(float)
            temperature = data[:,5].astype(float)#
            
        else: 
            print('The file format is unknown, or older. If it is an older file format, you will have to change the number of columns in the CW class.')
            
            
        # Simple check to see if the events are sequential
        def sequential(l):
            l = np.asarray(l).astype(int) #replaced np.int with int
            check = range(min(l),max(l)+1)

            if len(l)!=len(check):
                print('There is an event missing in the data.')
                return False

            counter = 0
            for i in range(len(l)):
                counter+=1
                if l[i]!=check[i]:
                    print('Check event number: '+str(counter))
            return sum(check == l)==len(l)

        if not sequential(event_number):
            print('Events in file are not sequential.')

        # Convert the computer time to an absolute time (MJD).
        if file_from_computer:
            time_stamp = []
            for i in range(len(comp_date)):
                year  = int(comp_date[i].split('-')[0])
                month = int(comp_date[i].split('-')[1])
                day   = int(comp_date[i].split('-')[2])
                hour  = int(comp_time[i].split(':')[0])
                mins  = int(comp_time[i].split(':')[1])
                sec   = int(np.floor(float(comp_time[i].split(':')[2])))
                try:  
                    decimal = float('0.'+str(comp_time[i].split('.')[-1]))
                except:
                    decimal = 0.0
                time_stamp.append(float(time.mktime((year, month, day, hour, mins, sec, 0, 0, 0)))+ decimal) 


            self.time_stamp_s     = np.asarray(time_stamp) -  min(np.asarray(time_stamp))       # The absolute time of an event in seconds
            self.time_stamp_ms    = np.asarray(time_stamp -  min(np.asarray(time_stamp)))*1000  # The absolute time of an event in miliseconds   
            self.total_time_s     = max(time_stamp) -  min(time_stamp)     # The absolute time of an event in seconds
            #self.detector_name    = detName                                
            #self.n_detector       = len(set(detName))

        # Convert the cumulative deadtime to the deadtime between events
        # The detector starts at time 0, so append a zero.
        event_deadtime_ms = diff(np.append([0],deadtime))

        # The Arduino absolute time isn't great. Over the course of a few hours, it will be off by several seconds. 
        # The computer will give you accurate time down to about 1ms. Reading from the serial port has ~ms scale uncertainty.
        # The Arduino can give you a precise measurement (down to 1us), but the absolute time will drift. Expect it to be off by roughly 1min per day.
        self.Ardn_time_ms      = Ardn_time_ms
        self.Ardn_time_s       = Ardn_time_ms/1000.
        
        self.Ardn_total_time_s = max(self.Ardn_time_s)
        self.Ardn_total_time_ms= max(self.Ardn_time_s)*1000.

        self.event_number     = np.asarray(event_number)  # an arrray of the event numbers
        self.total_counts     = max(event_number.astype(int)) - min(event_number.astype(int))
        self.adc              = adc         # an arrray of the measured event ADC value
        self.sipm             = sipm        # an arrray of the measured event SiPM value
        
        self.event_deadtime_s   = event_deadtime_ms/1000.      # an array of the measured event deadtime in seconds
        self.event_deadtime_ms  = event_deadtime_ms            # an array of the measured event deadtime in miliseconds
        self.total_deadtime_ms  = max(event_deadtime_ms)       # an array of the measured event deadtime in miliseconds
        self.total_deadtime_s   = max(event_deadtime_ms)/1000. # The total deadtime in seconds
                
        # The time between events is well described by the Arduino timestamp. 
        # The 'diff' command takes the difference between each element in the array.
        self.Ardn_event_livetime_s = diff(np.append([0],self.Ardn_time_s)) - self.event_deadtime_s
        

        if file_from_computer:
            self.live_time        = (self.total_time_s - self.total_deadtime_s)
            self.weights          = np.ones(len(event_number)) / self.live_time
            self.count_rate       = self.total_counts/self.live_time 
            self.count_rate_err   = np.sqrt(self.total_counts)/self.live_time 

            bins = range(0,int(max(self.time_stamp_s)), self.bin_size)
            counts, binEdges       = np.histogram(self.time_stamp_s, bins = bins)
            bin_livetime, binEdges = np.histogram(self.time_stamp_s, bins = bins, weights = self.Ardn_event_livetime_s)
        
        elif file_from_sdcard:
            self.live_time        = (self.Ardn_total_time_ms - self.total_deadtime_ms)/1000.
            self.weights          = np.ones(len(event_number)) / self.live_time
            self.count_rate       = self.total_counts/self.live_time 
            self.count_rate_err   = np.sqrt(self.total_counts)/self.live_time 

            bins = range(int(min(self.Ardn_time_s)),int(max(self.Ardn_time_s)),self.bin_size)
            counts, binEdges = np.histogram(self.Ardn_time_s, bins = bins)
            bin_livetime, binEdges = np.histogram(self.Ardn_time_s, bins = bins, weights = self.Ardn_event_livetime_s)
            
        else:
            print('Error')
        
        print('Count rate: '+str(np.round(self.count_rate,4)) +' +/- '+ str(np.round(self.count_rate_err,4)))
        
        bincenters = 0.5*(binEdges[1:]+ binEdges[:-1])
        
        self.bin_size          = bin_size
        self.binned_counts     = counts
        self.binned_counts_err = np.sqrt(counts)
        self.binned_count_rate = counts/bin_livetime
        self.binned_count_rate_err = np.sqrt(counts)/bin_livetime
        self.binned_time_s     = bincenters
        self.binned_time_m     = bincenters/60.

    
        
def plusSTD(n,array):
    xh = np.add(n,np.sqrt(np.abs(array)))
    return xh

def subSTD(n,array):
    xl = np.subtract(n,np.sqrt(np.abs(array)))
    return xl

def fill_between_steps(x, y1, y2=0, h_align='mid', ax=None,lw=2, **kwargs):
    # If no Axes opject given, grab the current one:
    if ax is None:
        ax = plt.gca()
    # First, duplicate the x values
    xx = x.repeat(2)[1:]
    # Now: the average x binwidth
    xstep = np.repeat((x[1:] - x[:-1]), 2) #swap sp for np
    xstep = np.concatenate(([xstep[0]], xstep, [xstep[-1]])) #swap sp for np
    # Now: add one step at end of row.
    xx = np.append(xx, xx.max() + xstep[-1])

    # Make it possible to chenge step alignment.
    if h_align == 'mid':
        xx -= xstep / 2.
    elif h_align == 'right':
        xx -= xstep

    # Also, duplicate each y coordinate in both arrays
    y1 = y1.repeat(2)#[:-1]
    if type(y2) == np.ndarray: ##sp to np
        y2 = y2.repeat(2)#[:-1]

    # now to the plotting part:
    ax.fill_between(xx, y1, y2=y2,lw=lw, **kwargs)
    return ax

print('Definitions complete ...')

# Shielding Multiple histogram 

In [ ]:
class NPlot():
    def __init__(self, 
                 data,
                 weights,
                 colors,
                 labels,
                 xmin,xmax,ymin,ymax,
                 figsize = [10,10],fontsize = 18,nbins = 101, alpha = 0.9,
                 xscale = 'log',yscale = 'log',xlabel = '',loc = 1,pdf_name='',lw=2, title=''):
        
        fig = plt.figure(figsize=(figsize[0], figsize[1])) 
        ax1 = fig.add_subplot(111)

        if xscale == 'log':
            bins = np.logspace(np.log10(xmin),np.log10(xmax),nbins)
        if xscale == 'linear':
            bins = np.linspace(xmin,xmax,nbins)
    
        for i in range(len(data)):
            counts,binEdges = np.histogram(data[i][~np.isnan(data[i])],bins = bins,weights = weights[i][~np.isnan(weights[i])],range= (200,1024))
            bincenters = 0.5*(binEdges[1:]+ binEdges[:-1])
            sumWeightsSqrd, binEdges = np.histogram(data[i][~np.isnan(data[i])], bins = bins, weights = np.power(weights[i][~np.isnan(weights[i])],2),range= (200,1024))
            fill_between_steps(bincenters, plusSTD(counts,sumWeightsSqrd),subSTD(counts,sumWeightsSqrd),  color = colors[i],alpha = alpha,lw=lw)
            pylab.plot([1e14,1e14], label = labels[i],color = colors[i],alpha = alpha,linewidth = 2)
        
        pylab.yscale(yscale) #removed nonposy='clip
        pylab.xscale(xscale) #removed nonposx
        pylab.grid(True, which='major', color='k', alpha = 0.1,linestyle='-')
        pylab.grid(True, which='minor', color='k',alpha = .05,  linestyle='-')
        pylab.legend(fontsize=fontsize-2,loc = loc,  fancybox = True,frameon=True)
        pylab.ylabel(r'Rate/bin [s$^{-1}$]',size=fontsize)
        pylab.xlabel(xlabel, labelpad=20,size=fontsize)
        pylab.axis([xmin, xmax, ymin,ymax])
        plt.title(title,fontsize=fontsize)
        pylab.tight_layout()
        if pdf_name != '':
            plt.savefig(pdf_name, format='pdf',transparent =True)
        pylab.show()


In [ ]:
# Load data that was taken from a computer
#    1. Give the name of the file
#    2. Give the size of binning in time you would like
import os
cwd = os.getcwd()
f1 = CWClass(cwd + "/baseline.txt", bin_size = 60)
f2 = CWClass(cwd + "/material1.txt", bin_size = 60)
f3 = CWClass(cwd + "/material2.txt", bin_size = 60)
f4 = CWClass(cwd + "/material3.txt", bin_size = 60)
f5 = CWClass(cwd + "/material4.txt", bin_size = 60)
f6 = CWClass(cwd + "/material5.txt", bin_size = 60)

In [ ]:
# Plot data
c = NPlot(
    data=[f1.sipm, f2.sipm, f3.sipm, f4.sipm, f5.sipm, f6.sipm],
    weights=[f1.weights, f2.weights, f3.weights, f4.weights, f5.weights, f6.weights],
    colors=[mycolors[1], mycolors[2], mycolors[3], mycolors[4], mycolors[5], mycolors[6]],
    labels=[
        r'Baseline',
        r'Material 1',
        r'Material 2',
        r'Material 3',
        r'Material 4',
        r'Material 5'
    ],
    figsize=[10, 6],
    fontsize=22,
    xmin=10, xmax=1400,
    ymin=1e-6, ymax=1,
    nbins=51, alpha=1, lw=2,
    xscale='log', yscale='log',
    xlabel='Calculated SiPM peak voltage [mV]',
    title='Multiple Histogram Plot',
    loc=1, pdf_name=''
)


interpretation of plot:
Shielding measurement: measure the effectiveness of a shield at reducing the strength of an interfering signal

# Shielding Rate plot

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class ratePlot:
    def __init__(self,
                 time,
                 count_rates,
                 count_rates_err,
                 colors,
                 labels,
                 xmin, xmax, ymin, ymax,
                 figsize=(10, 10), fontsize=18, alpha=0.9,
                 xscale='linear', yscale='linear',
                 xlabel='', ylabel='',
                 loc=1, pdf_name='', title=''):

        plt.rcParams.update({'font.size': fontsize})

        fig, ax = plt.subplots(figsize=figsize)

        for i in range(len(count_rates)):
            yerr = np.abs(count_rates_err[i])  # Ensure no negative errors

            ax.errorbar(time[i],
                        count_rates[i],
                        xerr=0,
                        yerr=yerr,
                        fmt='o',
                        label=labels[i],
                        linewidth=2,
                        ecolor=colors[i],
                        markersize=4,
                        color=colors[i],
                        alpha=alpha)

        ax.set_yscale(yscale)
        ax.set_xscale(xscale)
        ax.set_ylabel(ylabel)
        ax.set_xlabel(xlabel)
        ax.set_xlim([xmin, xmax])
        ax.set_ylim([ymin, ymax])
        ax.xaxis.labelpad = 0
        ax.legend(loc=loc, shadow=True, fontsize=fontsize, frameon=False, fancybox=True)
        ax.set_title(title)

        plt.tight_layout()
        if pdf_name:
            plt.savefig(pdf_name, format='pdf', transparent=True)
        plt.show()


In [ ]:
c = ratePlot(time = [f1.binned_time_m,f2.binned_time_m, f3.binned_time_m,f4.binned_time_m, f5.binned_time_m,f6.binned_time_m],
             count_rates = [f1.binned_count_rate,f2.binned_count_rate, f3.binned_count_rate,f4.binned_count_rate, f5.binned_count_rate,f6.binned_count_rate],
             count_rates_err = [f1.binned_count_rate_err,f2.binned_count_rate_err, f3.binned_count_rate_err,f4.binned_count_rate_err, f5.binned_count_rate_err,f6.binned_count_rate_err],
             colors = [mycolors[1],mycolors[2],mycolors[3],mycolors[4],mycolors[5],mycolors[6]],
             labels=[r'Baseline',r'Material 1',r'Material 2', r'Material 3',r'Material 4',r'Material 5'],
             xmin = 0,xmax = 20,ymin = 0,ymax = 0.55,
             figsize = [12,6],fontsize = 20,alpha = 1,
             xscale = 'linear',yscale = 'linear',xlabel = 'Time [min]',ylabel = r'Rate [s$^{-1}$]',
             loc = 1,pdf_name='',title = 'Rate Measurement')
